<a href="https://colab.research.google.com/github/gift-framework/GIFT/blob/research/research/notebooks/K7_Riemann_Verification_v3_Spectral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K₇ Riemann Verification v3.1 — Spectral Method (CuPy)

## Fourier-Galerkin sur 7-Tore avec Métrique G₂

**Problème v3.0**: PINN collapse → fonctions constantes (λ→0)

**Solution v3.1**: Méthode spectrale exacte
1. Base de Fourier sur T⁷ = [0,2π]⁷
2. Laplacien pondéré par métrique G₂: Δ_g = g^{ij}∂ᵢ∂ⱼ
3. Diagonalisation directe (petite base) ou itérative (grande base)

**Avantage**: Eigenvalues EXACTS pour la base tronquée (pas d'approximation variationnelle)

---

**Hypothesis**: γₙ = λₙ × H* (Riemann zeros encode K₇ spectrum)

**Requirements**: CuPy, CUDA GPU (A100 recommended)

**Author**: GIFT Framework | **Date**: 2026-01-30 | **Version**: 3.1

In [ ]:
# ============================================================
# CELL 1: Setup & GPU Detection
# ============================================================
import subprocess
import sys
import time
import warnings
warnings.filterwarnings('ignore')

def install(pkg):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

import numpy as np
from typing import Tuple, List, Dict
from itertools import product

# CuPy setup
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU = True
    props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = props['name'].decode() if isinstance(props['name'], bytes) else props['name']
    gpu_mem = cp.cuda.Device().mem_info[1] / 1e9
    print(f"✓ GPU ENABLED: {gpu_name}")
    print(f"  Memory: {gpu_mem:.1f} GB")
except ImportError:
    print("Installing CuPy...")
    install('cupy-cuda12x')
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU = True

import scipy.sparse as sp
import scipy.sparse.linalg as sp_linalg

print(f"\n✓ Setup complete | NumPy {np.__version__} | CuPy {cp.__version__}")

In [ ]:
# ============================================================
# CELL 2: GIFT Constants & Riemann Zeros
# ============================================================

class GIFTConstants:
    """Topological constants from K₇ manifold with G₂ holonomy."""
    dim_K7 = 7
    b2 = 21
    b3 = 77
    H_star = 99
    dim_G2 = 14
    dim_E8 = 248
    rank_E8 = 8
    
    # Metric
    det_g = 65/32
    metric_scale = (det_g) ** (1/dim_K7)
    
    # Prediction
    lambda1_pred = dim_G2 / H_star

G = GIFTConstants()

# Riemann zeros
RIEMANN_ZEROS = np.array([
    14.134725142, 21.022039639, 25.010857580, 30.424876126, 32.935061588,
    37.586178159, 40.918719012, 43.327073281, 48.005150881, 49.773832478,
    52.970321478, 56.446247697, 59.347044003, 60.831778525, 65.112544048,
    67.079810529, 69.546401711, 72.067157674, 75.704690699, 77.144840069,
    79.337375020, 82.910380854, 84.735492981, 87.425274613, 88.809111208,
    92.491899271, 94.651344041, 95.870634228, 98.831194218, 101.317851006,
], dtype=np.float64)

RIEMANN_RATIOS = RIEMANN_ZEROS / RIEMANN_ZEROS[0]

print("="*65)
print("GIFT CONSTANTS")
print("="*65)
print(f"  H* = {G.H_star}, dim(G₂) = {G.dim_G2}")
print(f"  λ₁ predicted = {G.lambda1_pred:.6f}")
print(f"  First 5 Riemann ratios: {RIEMANN_RATIOS[:5].round(4)}")

In [ ]:
# ============================================================
# CELL 3: G₂ Metric Construction
# ============================================================

def build_g2_metric():
    """
    Build G₂ metric from standard 3-form φ₀.
    
    φ₀ = e¹²³ + e¹⁴⁵ + e¹⁶⁷ + e²⁴⁶ - e²⁵⁷ - e³⁴⁷ - e³⁵⁶
    g_ij = (1/6) Σ_{kl} φ_ikl φ_jkl
    
    Returns: g_ij (7x7) and g^{ij} (inverse)
    """
    # Standard G₂ 3-form coefficients
    phi_terms = [
        ((0,1,2), +1), ((0,3,4), +1), ((0,5,6), +1),
        ((1,3,5), +1), ((1,4,6), -1), ((2,3,6), -1), ((2,4,5), -1)
    ]
    
    phi = np.zeros((7,7,7), dtype=np.float64)
    for (i,j,k), sign in phi_terms:
        # Antisymmetric
        phi[i,j,k] = phi[j,k,i] = phi[k,i,j] = sign
        phi[j,i,k] = phi[i,k,j] = phi[k,j,i] = -sign
    
    # Metric from 3-form: g_ij = (1/6) φ_ikl φ_jkl
    g = np.einsum('ikl,jkl->ij', phi, phi) / 6.0
    
    # Scale to match GIFT det(g) = 65/32
    current_det = np.linalg.det(g)
    scale = (G.det_g / current_det) ** (1/7)
    g_scaled = g * (scale ** 2)
    
    g_inv = np.linalg.inv(g_scaled)
    
    return g_scaled, g_inv

G_METRIC, G_INV = build_g2_metric()

print("="*65)
print("G₂ METRIC")
print("="*65)
print(f"  g diagonal: {np.diag(G_METRIC).round(4)}")
print(f"  det(g) = {np.linalg.det(G_METRIC):.6f} (target: {G.det_g:.6f})")
print(f"  g^{'{'}ij{'}'} diagonal: {np.diag(G_INV).round(4)}")

In [ ]:
# ============================================================
# CELL 4: Fourier Basis on 7-Torus
# ============================================================

def generate_fourier_modes(k_max: int) -> np.ndarray:
    """
    Generate Fourier mode indices for 7-torus.
    
    Modes: e^{i k·x} where k ∈ ℤ⁷, |k|² ≤ k_max²
    
    Returns: array of shape (N_modes, 7) with integer wave vectors
    """
    # Generate all combinations in [-k_max, k_max]^7
    ranges = [range(-k_max, k_max+1) for _ in range(7)]
    
    modes = []
    for k in product(*ranges):
        k = np.array(k)
        # Filter by |k|² ≤ k_max² (ball, not cube)
        if np.sum(k**2) <= k_max**2:
            modes.append(k)
    
    modes = np.array(modes)
    
    # Sort by |k|² (eigenvalue ordering)
    norms_sq = np.sum(modes**2, axis=1)
    order = np.argsort(norms_sq)
    modes = modes[order]
    
    return modes

def laplacian_eigenvalue_flat(k: np.ndarray) -> float:
    """
    Eigenvalue of flat Laplacian for mode k.
    -Δ e^{ik·x} = |k|² e^{ik·x}
    """
    return np.sum(k**2)

def laplacian_eigenvalue_g2(k: np.ndarray, g_inv: np.ndarray) -> float:
    """
    Eigenvalue of G₂-weighted Laplacian for mode k.
    -Δ_g e^{ik·x} = (g^{ij} k_i k_j) e^{ik·x}
    """
    return k @ g_inv @ k

# Test
print("="*65)
print("FOURIER BASIS GENERATION")
print("="*65)

for k_max in [1, 2, 3]:
    modes = generate_fourier_modes(k_max)
    print(f"  k_max = {k_max}: {len(modes)} modes")

In [ ]:
# ============================================================
# CELL 5: Compute K₇ Spectrum (G₂ Laplacian)
# ============================================================

def compute_k7_spectrum(k_max: int, n_eigenvalues: int = 30) -> Tuple[np.ndarray, np.ndarray]:
    """
    Compute first n eigenvalues of G₂-weighted Laplacian on T⁷.
    
    For Fourier modes, the Laplacian is DIAGONAL:
    λ(k) = g^{ij} k_i k_j
    
    Returns:
        eigenvalues: sorted array of λ values
        modes: corresponding wave vectors
    """
    print(f"\n  Generating Fourier modes (k_max={k_max})...")
    modes = generate_fourier_modes(k_max)
    n_modes = len(modes)
    print(f"  Total modes: {n_modes}")
    
    # Compute eigenvalues (diagonal in Fourier basis)
    print(f"  Computing G₂-weighted eigenvalues...")
    eigenvalues = np.array([laplacian_eigenvalue_g2(k, G_INV) for k in modes])
    
    # Sort by eigenvalue
    order = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[order]
    modes = modes[order]
    
    # Skip zero mode (k=0 → λ=0, constant function)
    nonzero_mask = eigenvalues > 1e-10
    eigenvalues = eigenvalues[nonzero_mask]
    modes = modes[nonzero_mask]
    
    print(f"  Non-zero eigenvalues: {len(eigenvalues)}")
    
    # Return first n
    n = min(n_eigenvalues, len(eigenvalues))
    return eigenvalues[:n], modes[:n]

print("="*65)
print("K₇ SPECTRUM (G₂ Laplacian on T⁷)")
print("="*65)

# Compute spectrum
k_max = 4  # Gives ~3000 modes, enough for 30 eigenvalues
eigenvalues_raw, modes = compute_k7_spectrum(k_max, n_eigenvalues=30)

print(f"\n  First 10 raw eigenvalues: {eigenvalues_raw[:10].round(4)}")
print(f"  First 10 modes:")
for i in range(min(10, len(modes))):
    print(f"    k_{i+1} = {modes[i]}, λ = {eigenvalues_raw[i]:.4f}")

In [ ]:
# ============================================================
# CELL 6: Normalize to Match γ₁ = 14.135
# ============================================================

# The raw eigenvalues are for the flat torus scaled by G₂ metric.
# We need to find the correct normalization such that:
#   λ₁ × H* = γ₁ = 14.135
# or equivalently: λ₁ = γ₁/H* = 14.135/99 ≈ 0.1428

# Method: Find scale factor c such that c × λ₁_raw = γ₁/H*
lambda1_target = RIEMANN_ZEROS[0] / G.H_star
scale_factor = lambda1_target / eigenvalues_raw[0]

# Normalized eigenvalues
eigenvalues = eigenvalues_raw * scale_factor

# Predicted gamma values
gamma_pred = eigenvalues * G.H_star

print("="*65)
print("EIGENVALUE NORMALIZATION")
print("="*65)
print(f"\n  Raw λ₁ = {eigenvalues_raw[0]:.6f}")
print(f"  Target λ₁ = γ₁/H* = {lambda1_target:.6f}")
print(f"  Scale factor = {scale_factor:.6f}")
print(f"\n  Normalized λ₁ = {eigenvalues[0]:.6f}")
print(f"  γ₁ = λ₁ × H* = {gamma_pred[0]:.4f} (target: {RIEMANN_ZEROS[0]:.4f})")

In [ ]:
# ============================================================
# CELL 7: Spectral Ratio Analysis
# ============================================================

n_eigs = len(eigenvalues)
gamma_actual = RIEMANN_ZEROS[:n_eigs]

# Compute ratios
ratios_pred = eigenvalues / eigenvalues[0]
ratios_actual = RIEMANN_RATIOS[:n_eigs]

# Deviations
gamma_dev = np.abs(gamma_pred - gamma_actual) / gamma_actual * 100
ratio_dev = np.abs(ratios_pred - ratios_actual) / ratios_actual * 100

print("="*80)
print("SPECTRAL RATIO COMPARISON: K₇ vs RIEMANN")
print("="*80)
print(f"\n{'n':>3} | {'λₙ':>10} | {'γₙ(K₇)':>10} | {'γₙ(Riem)':>10} | {'λₙ/λ₁':>8} | {'γₙ/γ₁':>8} | {'Δ%':>6}")
print("-"*75)

for i in range(n_eigs):
    n = i + 1
    status = '★★★' if ratio_dev[i] < 1 else '★★' if ratio_dev[i] < 5 else '★' if ratio_dev[i] < 10 else ''
    print(f"{n:3} | {eigenvalues[i]:10.6f} | {gamma_pred[i]:10.4f} | {gamma_actual[i]:10.4f} | "
          f"{ratios_pred[i]:8.4f} | {ratios_actual[i]:8.4f} | {ratio_dev[i]:5.2f}% {status}")

# Summary
matches_1 = np.sum(ratio_dev < 1)
matches_5 = np.sum(ratio_dev < 5)
matches_10 = np.sum(ratio_dev < 10)

print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}")
print(f"  Ratio matches < 1%:  {matches_1}/{n_eigs}")
print(f"  Ratio matches < 5%:  {matches_5}/{n_eigs}")
print(f"  Ratio matches < 10%: {matches_10}/{n_eigs}")
print(f"  Mean ratio deviation: {np.mean(ratio_dev):.2f}%")

In [ ]:
# ============================================================
# CELL 8: Analyze Eigenvalue Degeneracies
# ============================================================

# On a torus with G₂ metric, many eigenvalues are degenerate
# (same |k|² but different k vectors)

# Let's count multiplicities
from collections import Counter

# Round to detect degeneracies
eig_rounded = np.round(eigenvalues_raw, decimals=6)
multiplicities = Counter(eig_rounded)

print("="*65)
print("EIGENVALUE DEGENERACIES")
print("="*65)

unique_eigs = sorted(multiplicities.keys())
print(f"\n  Distinct eigenvalue levels: {len(unique_eigs)}")
print(f"\n  First 15 levels with multiplicities:")

for i, eig in enumerate(unique_eigs[:15]):
    mult = multiplicities[eig]
    scaled_eig = eig * scale_factor
    gamma = scaled_eig * G.H_star
    print(f"    Level {i+1}: λ = {scaled_eig:.4f}, γ = {gamma:.2f}, mult = {mult}")

# This reveals that the torus spectrum is highly degenerate
# The first non-trivial level has multiplicity 14 (= dim(G₂)!)
print(f"\n  First level multiplicity: {multiplicities[unique_eigs[0]]}")
print(f"  (Note: dim(G₂) = 14)")

In [ ]:
# ============================================================
# CELL 9: Use Distinct Eigenvalue LEVELS (not counting mult)
# ============================================================

# The correct comparison might be with DISTINCT eigenvalue levels
# Not with repeated eigenvalues

unique_levels = sorted(set(np.round(eigenvalues_raw, 6)))
eigenvalues_levels = np.array(unique_levels) * scale_factor

n_levels = min(30, len(eigenvalues_levels))
eigenvalues_levels = eigenvalues_levels[:n_levels]

gamma_levels = eigenvalues_levels * G.H_star
ratios_levels = eigenvalues_levels / eigenvalues_levels[0]

gamma_actual_30 = RIEMANN_ZEROS[:n_levels]
ratios_actual_30 = gamma_actual_30 / gamma_actual_30[0]

ratio_dev_levels = np.abs(ratios_levels - ratios_actual_30) / ratios_actual_30 * 100

print("="*80)
print("DISTINCT LEVEL COMPARISON: K₇ vs RIEMANN")
print("="*80)
print(f"\n{'Level':>5} | {'λ_level':>10} | {'γ(K₇)':>10} | {'γ(Riem)':>10} | {'ratio_K7':>8} | {'ratio_R':>8} | {'Δ%':>6}")
print("-"*80)

for i in range(n_levels):
    status = '★★★' if ratio_dev_levels[i] < 1 else '★★' if ratio_dev_levels[i] < 5 else '★' if ratio_dev_levels[i] < 10 else ''
    print(f"{i+1:5} | {eigenvalues_levels[i]:10.6f} | {gamma_levels[i]:10.4f} | {gamma_actual_30[i]:10.4f} | "
          f"{ratios_levels[i]:8.4f} | {ratios_actual_30[i]:8.4f} | {ratio_dev_levels[i]:5.2f}% {status}")

matches_1_lev = np.sum(ratio_dev_levels < 1)
matches_5_lev = np.sum(ratio_dev_levels < 5)
matches_10_lev = np.sum(ratio_dev_levels < 10)

print(f"\n{'='*80}")
print(f"  Level ratio matches < 1%:  {matches_1_lev}/{n_levels}")
print(f"  Level ratio matches < 5%:  {matches_5_lev}/{n_levels}")
print(f"  Level ratio matches < 10%: {matches_10_lev}/{n_levels}")
print(f"  Mean level ratio deviation: {np.mean(ratio_dev_levels):.2f}%")

In [ ]:
# ============================================================
# CELL 10: Visualization
# ============================================================
try:
    import matplotlib.pyplot as plt
    HAS_MPL = True
except ImportError:
    HAS_MPL = False

if HAS_MPL:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Spectral Ratios (all eigenvalues)
    ax = axes[0, 0]
    n_plot = min(30, len(ratios_pred))
    ax.plot(range(1, n_plot+1), ratios_actual[:n_plot], 'ko-', lw=2, ms=6, label='Riemann γₙ/γ₁')
    ax.plot(range(1, n_plot+1), ratios_pred[:n_plot], 'bs--', lw=2, ms=5, alpha=0.7, label='K₇ λₙ/λ₁')
    ax.set_xlabel('Index n')
    ax.set_ylabel('Spectral Ratio')
    ax.set_title('Spectral Ratios (with degeneracy)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. Spectral Ratios (distinct levels)
    ax = axes[0, 1]
    n_lev_plot = min(30, len(ratios_levels))
    ax.plot(range(1, n_lev_plot+1), ratios_actual_30[:n_lev_plot], 'ko-', lw=2, ms=6, label='Riemann γₙ/γ₁')
    ax.plot(range(1, n_lev_plot+1), ratios_levels[:n_lev_plot], 'rs--', lw=2, ms=5, alpha=0.7, label='K₇ levels')
    ax.set_xlabel('Level n')
    ax.set_ylabel('Spectral Ratio')
    ax.set_title('Distinct Eigenvalue Levels')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 3. Ratio deviation histogram
    ax = axes[1, 0]
    ax.hist(ratio_dev_levels, bins=15, edgecolor='black', alpha=0.7, color='steelblue')
    ax.axvline(5, color='orange', ls='--', lw=2, label='5% threshold')
    ax.axvline(10, color='red', ls='--', lw=2, label='10% threshold')
    ax.set_xlabel('Ratio Deviation %')
    ax.set_ylabel('Count')
    ax.set_title('Distribution of Ratio Deviations')
    ax.legend()
    
    # 4. γ predicted vs actual
    ax = axes[1, 1]
    ax.scatter(gamma_actual_30, gamma_levels, c=ratio_dev_levels, cmap='RdYlGn_r', 
               s=80, edgecolors='black')
    ax.plot([10, 110], [10, 110], 'k--', lw=2)
    ax.set_xlabel('γₙ (Riemann)')
    ax.set_ylabel('γₙ (K₇)')
    ax.set_title('γₙ Correspondence')
    cbar = plt.colorbar(ax.collections[0], ax=ax)
    cbar.set_label('Deviation %')
    
    plt.tight_layout()
    plt.savefig('K7_Riemann_v3_spectral_results.png', dpi=150)
    plt.show()
    print("\n✓ Figure saved: K7_Riemann_v3_spectral_results.png")

In [ ]:
# ============================================================
# CELL 11: Weyl Law Analysis
# ============================================================

# Weyl's law for d-dimensional manifold:
# N(λ) ~ c_d × Vol × λ^{d/2}
# For d=7: N(λ) ~ λ^{3.5}
# So λ_n ~ n^{2/7} ≈ n^{0.286}

n_vals = np.arange(1, len(eigenvalues_levels)+1)
weyl_pred = eigenvalues_levels[0] * n_vals ** (2/7)

# Riemann zeros grow like:
# γ_n ~ 2πn / log(n)
# which is roughly linear for small n

print("="*65)
print("WEYL LAW ANALYSIS")
print("="*65)
print(f"\n  Expected growth: λₙ ~ n^(2/7) ≈ n^0.286")
print(f"  Riemann growth: γₙ ~ 2πn/log(n) (asymptotic)")

# Fit power law to K₇ eigenvalues
log_n = np.log(n_vals[1:])  # Skip n=1 to avoid log(1)=0
log_lambda = np.log(eigenvalues_levels[1:])
slope, intercept = np.polyfit(log_n, log_lambda, 1)

print(f"\n  Fitted K₇ growth: λₙ ~ n^{slope:.3f}")
print(f"  Weyl prediction: λₙ ~ n^{2/7:.3f}")
print(f"  Deviation: {abs(slope - 2/7)/(2/7)*100:.1f}%")

# Fit Riemann zeros
log_gamma = np.log(RIEMANN_ZEROS[1:len(eigenvalues_levels)])
slope_r, _ = np.polyfit(log_n, log_gamma, 1)
print(f"\n  Fitted Riemann growth: γₙ ~ n^{slope_r:.3f}")

In [ ]:
# ============================================================
# CELL 12: Final Analysis & Export
# ============================================================
import json

print("="*80)
print("█" + " "*30 + "FINAL RESULTS" + " "*31 + "█")
print("="*80)

print(f"""
┌──────────────────────────────────────────────────────────────────────────────┐
│  K₇ RIEMANN VERIFICATION v3.1 — SPECTRAL METHOD                              │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  METHOD: Fourier-Galerkin on T⁷ with G₂ metric                               │
│  MODES: k_max = {k_max}, total = {len(modes)} Fourier modes                            │
│                                                                              │
│  KEY FINDING: First eigenvalue level has multiplicity {multiplicities[unique_eigs[0]]:2d}                 │
│               This equals dim(G₂) = 14 (!) — structural connection           │
│                                                                              │
│  SPECTRAL RATIO RESULTS (distinct levels):                                   │
│    Matches < 5%:  {matches_5_lev:2d}/{n_levels:2d} levels                                            │
│    Matches < 10%: {matches_10_lev:2d}/{n_levels:2d} levels                                            │
│    Mean deviation: {np.mean(ratio_dev_levels):5.2f}%                                              │
│                                                                              │
│  WEYL EXPONENT: K₇ gives λₙ ~ n^{slope:.3f} (theory: n^{2/7:.3f})                    │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
""")

# Key insight
print("""
┌──────────────────────────────────────────────────────────────────────────────┐
│  KEY INSIGHT                                                                 │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  The TORUS T⁷ with G₂ metric is NOT the same as the compact K₇ manifold.    │
│                                                                              │
│  The torus has:                                                              │
│    - High degeneracy (first level: mult = 14)                                │
│    - Weyl growth ~ n^{2/7} (correct for d=7)                                 │
│    - But spectral ratios DON'T match Riemann                                 │
│                                                                              │
│  This suggests:                                                              │
│    1. K₇ needs TOPOLOGY (twisted connected sum), not just metric             │
│    2. The non-trivial π₁ and Betti numbers affect spectrum                   │
│    3. Or: Riemann zeros encode a DIFFERENT spectral object                   │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
""")

# Export
results = {
    'version': '3.1',
    'method': 'Fourier-Galerkin on T7 with G2 metric',
    'k_max': int(k_max),
    'n_modes': int(len(modes)),
    'eigenvalues_raw': eigenvalues_raw.tolist(),
    'eigenvalues_normalized': eigenvalues.tolist(),
    'eigenvalue_levels': eigenvalues_levels.tolist(),
    'gamma_predicted': gamma_levels.tolist(),
    'gamma_actual': gamma_actual_30.tolist(),
    'ratio_deviations': ratio_dev_levels.tolist(),
    'first_level_multiplicity': int(multiplicities[unique_eigs[0]]),
    'weyl_exponent_fitted': float(slope),
    'summary': {
        'matches_5pct': int(matches_5_lev),
        'matches_10pct': int(matches_10_lev),
        'mean_deviation': float(np.mean(ratio_dev_levels)),
    }
}

with open('K7_Riemann_v3_spectral_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✓ Results saved: K7_Riemann_v3_spectral_results.json")

## Next Steps for v4

### The Torus ≠ K₇ Problem

The 7-torus T⁷ with G₂ metric is **topologically different** from the compact G₂ manifold K₇:

| Property | T⁷ | K₇ (Joyce/Kovalev) |
|----------|-----|--------------------|
| π₁ | ℤ⁷ | trivial |
| b₂ | 21 | 21 ✓ |
| b₃ | 35 | **77** ✗ |
| Holonomy | SU(3) ⊂ G₂ | full G₂ |

### Possible Approaches for v4

1. **TCS Construction**: Implement twisted connected sum numerically
   - Glue two ACyl Calabi-Yau 3-folds × S¹
   - Compute spectrum on the connected sum

2. **Deformation from Torus**: Add G₂ torsion terms to break degeneracy
   - The torsion-free condition gives extra constraints

3. **Alternative Hypothesis**: γₙ may encode:
   - Hodge Laplacian on forms (not scalar Laplacian)
   - Dirac operator spectrum
   - Selberg zeta zeros (length spectrum of geodesics)

4. **Statistical Test**: Check if Riemann zero spacings match K₇ level spacings
   - GUE statistics for Riemann
   - GOE/Poisson for K₇?